In [1]:
import numpy as np
from collections import defaultdict

In [2]:
def read_dataset(path, labeled=True):
    with open(path, encoding="utf8") as fp:
        data = []
        sentence = []
        for line in fp:
            if line == "\n":
                # start again
                data.append(sentence)
                sentence = []
            else:
                if labeled:
                    tokens = line.strip().split()
                    sentence.append((' '.join(tokens[:-1]), tokens[-1]))
                else:
                    sentence.append((line.strip(), ))
    return data

In [3]:
def write_dataset(dataset, path):
    with open(path, 'w') as fp:
        for sentence in dataset:
            for row in sentence:
                fp.write(' '.join(row) + "\n")
            fp.write("\n")

# 2.1: 
Write a function that estimates the emission parameters from the training set using MLE (maximum
likelihood estimation)

In [4]:
def get_e_without_smoothing(dataset):
    # ys = set([y for sentence in dataset for (x, y) in sentence])
    words = set([x for sentence in dataset for (x, y) in sentence])
    count_ys = defaultdict(int)
    count_y_x = defaultdict(lambda  : defaultdict(int))
    for sentence in dataset:
        for word, label in sentence:
            count_ys[label] += 1
            count_y_x[label][word] += 1
    #es = {label: {x: count/count_ys[label] for x, count in emissions.iteritems()} for label, emissions in count_y_x.iteritems()}
    def e(x, y):
        return count_y_x[y][x]/count_ys[y]
    return e

# 2.2:
Set k to 1, implement this fix into your function for computing the emission parameters.

In [5]:
def get_e(dataset, k=1):
    # ys = set([y for sentence in dataset for (x, y) in sentence])
    words = set([x for sentence in dataset for (x, y) in sentence])
    count_ys = defaultdict(int)
    count_y_x = defaultdict(lambda  : defaultdict(int))
    for sentence in dataset:
        for word, label in sentence:
            count_ys[label] += 1
            count_y_x[label][word] += 1
    #es = {label: {x: count/count_ys[label] for x, count in emissions.iteritems()} for label, emissions in count_y_x.iteritems()}
    def e(x, y):
        if x in words:
            return count_y_x[y][x]/(count_ys[y] + k)
        else:
            return k/(count_ys[y] + k)
    return e

# 2.3: 
Compare your outputs and the gold-standard outputs in dev.out
and report the precision, recall and F scores of such a baseline system for each dataset.

In [6]:
def simple_labeling(dataset, train_set):
    ys = set([y for sentence in train_set for (x, y) in sentence])
    e = get_e(train_set)
    output = []
    for sentence in dataset:
        labeled_sentence = []
        for word,  in sentence:
            max_p = 0
            label = None
            for y in ys:
                p = e(word, y)
                if p > max_p:
                    max_p = p
                    label = y
            labeled_sentence.append((word, label))
        output.append(labeled_sentence)
    return output

In [7]:
en_train = read_dataset('EN/train')
en_dev = read_dataset('EN/dev.in', labeled=False)
en_dev_labeled = simple_labeling(en_dev, en_train)
write_dataset(en_dev_labeled, "EN/dev.p2.out")

# EN
## Entity in gold data: 802
## Entity in prediction: 1148

## Correct Entity : 614
Entity  precision: 0.5348

Entity  recall: 0.7656

Entity  F: 0.6297

## Correct Entity Type : 448
Entity Type  precision: 0.3902

Entity Type  recall: 0.5586

Entity Type  F: 0.4595


In [8]:
fr_train = read_dataset('FR/train')
fr_dev = read_dataset('FR/dev.in', labeled=False)
fr_dev_labeled = simple_labeling(fr_dev, fr_train)
write_dataset(fr_dev_labeled, "FR/dev.p2.out")

# FR

## Entity in gold data: 238
## Entity in prediction: 1114

## Correct Entity : 186
Entity  precision: 0.1670

Entity  recall: 0.7815

Entity  F: 0.2751

## Correct Entity Type : 79
Entity Type  precision: 0.0709

Entity Type  recall: 0.3319

Entity Type  F: 0.1169

In [10]:
cn_train = read_dataset('CN/train')
cn_dev = read_dataset('CN/dev.in', labeled=False)
cn_dev_labeled = simple_labeling(cn_dev, cn_train)
write_dataset(cn_dev_labeled, "CN/dev.p2.out")

UnicodeEncodeError: 'charmap' codec can't encode character '\u300a' in position 0: character maps to <undefined>

# CN
## Entity in gold data: 1081
## Entity in prediction: 5161

## Correct Entity : 602
Entity  precision: 0.1166

Entity  recall: 0.5569

Entity  F: 0.1929

## Correct Entity Type : 354

Entity Type  precision: 0.0686

Entity Type  recall: 0.3275

Entity Type  F: 0.1134

In [11]:
sg_train = read_dataset('SG/train')
sg_dev = read_dataset('SG/dev.in', labeled=False)
sg_dev_labeled = simple_labeling(sg_dev, sg_train)
write_dataset(sg_dev_labeled, "SG/dev.p2.out")

UnicodeEncodeError: 'charmap' codec can't encode character '\u266b' in position 0: character maps to <undefined>

# SG
## Entity in gold data: 4092
## Entity in prediction: 12125

## Correct Entity : 2394
Entity  precision: 0.1974

Entity  recall: 0.5850

Entity  F: 0.2952

## Correct Entity Type : 1217
Entity Type  precision: 0.1004

Entity Type  recall: 0.2974

Entity Type  F: 0.1501

# 3.1: 
Write a function that estimates the transition parameters from the training set using MLE (maximum
likelihood estimation)

In [12]:
def get_q(dataset, k=1):
    qs = defaultdict(lambda  : defaultdict(int))
    count_ys = defaultdict(int)
    count_ys['START'] = len(dataset)
    count_ys['STOP'] = len(dataset)
    for sentence in dataset:
        n = len(sentence)
        for i in range(n):
            current_y = sentence[i][1]
            count_ys[current_y] += 1
            if i == 0:
                qs[current_y]['START'] += 1
            elif i == n-1:
                qs['STOP'][current_y] += 1
                qs[current_y][sentence[i-1][1]] += 1
            else:
                qs[current_y][sentence[i-1][1]] += 1
    def q(yi, yi_1):
        if qs[yi][yi_1] == 0:
            return k/(count_ys[yi_1] + k) # smoothing
        return qs[yi][yi_1]/(count_ys[yi_1] + k)
    return q

# 3.2:
Use the estimated transition and emission parameters, implement the Viterbi algorithm taught in class
to compute the following (for a sentence with n words)

In [13]:
import sys
import warnings
warnings.filterwarnings('ignore')

def hmm(sentence, q, e, ys):
    pi = [{}]
    for y in ys:
        pi[-1][y] = np.log(0)
    pi[-1]['START'] = 0
    
    n = len(sentence)
    for i in range(n):
        x = sentence[i][0]
        pi.append({})
        for y in ys:
            pi[-1][y] = max([p + np.log(q(y, u)) + np.log(e(x, y)) for u, p in pi[-2].items()])
    pi.append({})
    pi[-1]['STOP'] = max([pi[-2][u] + np.log(q('STOP', u)) for u in ys])
    # return pi
    labels = [None] * n
    next_ = 'STOP'
    for i in range(n, 0, -1):
        max_p = np.log(0)
        best_y = None
        for y in ys:
            lgp = pi[i][y] + np.log(q(next_, y))
            if lgp >= max_p:
                max_p = lgp
                best_y = y
        next_ = best_y
        labels[i - 1] = best_y
    return list(zip([word for word,  in sentence], labels))

def label_with_hmm(dataset, train_set, k=1, get_q=get_q, get_e=get_e):
    ys = set([y for sentence in train_set for (x, y) in sentence])
    q = get_q(train_set, k=k)
    e = get_e(train_set, k=k)
    return [hmm(sentence, q, e, ys) for sentence in dataset]

In [14]:
en_dev_labeled_hmm = label_with_hmm(en_dev, en_train)
write_dataset(en_dev_labeled_hmm, "EN/dev.p3.out")

# EN
## Entity in gold data: 802
## Entity in prediction: 956

## Correct Entity : 564
Entity  precision: 0.5900

Entity  recall: 0.7032

Entity  F: 0.6416

## Correct Entity Type : 451
Entity Type  precision: 0.4718

Entity Type  recall: 0.5623

Entity Type  F: 0.5131

In [15]:
fr_dev_labeled_hmm = label_with_hmm(fr_dev, fr_train)
write_dataset(fr_dev_labeled_hmm, "FR/dev.p3.out")

# FR
## Entity in gold data: 238
## Entity in prediction: 448

## Correct Entity : 137
Entity  precision: 0.3058

Entity  recall: 0.5756

Entity  F: 0.3994

## Correct Entity Type : 76
Entity Type  precision: 0.1696

Entity Type  recall: 0.3193

Entity Type  F: 0.2216

In [16]:
cn_dev_labeled_hmm = label_with_hmm(cn_dev, cn_train)
write_dataset(cn_dev_labeled_hmm, "CN/dev.p3.out")

UnicodeEncodeError: 'charmap' codec can't encode character '\u300a' in position 0: character maps to <undefined>

# CN
## Entity in gold data: 1081
## Entity in prediction: 1221

## Correct Entity : 456
Entity  precision: 0.3735

Entity  recall: 0.4218

Entity  F: 0.3962

## Correct Entity Type : 309
Entity Type  precision: 0.2531

Entity Type  recall: 0.2858

Entity Type  F: 0.2685

In [ ]:
sg_dev_labeled_hmm = label_with_hmm(sg_dev, sg_train)
write_dataset(sg_dev_labeled_hmm, "SG/dev.p3.out")

# SG
## Entity in gold data: 4092
## Entity in prediction: 4118

## Correct Entity : 1720
Entity  precision: 0.4177

Entity  recall: 0.4203

Entity  F: 0.4190

## Correct Entity Type : 1039
Entity Type  precision: 0.2523

Entity Type  recall: 0.2539

Entity Type  F: 0.2531

# 4.1:
Describe the Viterbi algorithm used for decoding such a second-order HMM model and implement it.

In [ ]:
def get_second_order_q(dataset, k=1):
    qs = defaultdict(lambda  : defaultdict(lambda: defaultdict(int))) # {yi: {yi_1: {yi_2: prob}}}
    count_ys = defaultdict(lambda: defaultdict(int)) # {yi_1: {yi_2: count}}
    count_ys['START']['START'] = len(dataset)
    
    for sentence in dataset:
        n = len(sentence)
        for i in range(n):
            current_y = sentence[i][1]
            if i == 0:
                count_ys[current_y]['START'] += 1
            else:
                prev_y = sentence[i-1][1]
                count_ys[current_y][prev_y] += 1
            
            if i == 0:
                qs[current_y]['START']['START'] += 1
            elif i == 1:
                qs[current_y][sentence[i-1][1]]['START'] += 1
            elif i == n-1:
                qs[current_y][sentence[i-1][1]][sentence[i-2][1]] += 1
                qs['STOP'][current_y][sentence[i-1][1]] += 1
            else:
                qs[current_y][sentence[i-1][1]][sentence[i-2][1]] += 1
                
    def q(yi, yi_1, yi_2):
        if qs[yi][yi_1][yi_2] == 0:
            if count_ys[yi_1][yi_2] == 0:
                return 0
            else:
                return k/(count_ys[yi_1][yi_2] + k) # smoothing
        return qs[yi][yi_1][yi_2]/(count_ys[yi_1][yi_2] + k)
    return q

In [ ]:
def second_order_hmm(sentence, q, e, ys):
    pi = [defaultdict(lambda: defaultdict(int)), defaultdict(lambda: defaultdict(int))] # array of {v: {u: prob}}
    pi[0]['START']['START'] = 1
    
    for y in ys:
        pi[1][y]['START'] = q(y, 'START', 'START') * e(sentence[0][0], y)
    
    n = len(sentence)
    
    for i in range(1, n):
        word = sentence[i][0]
        
        pi.append(defaultdict(lambda : defaultdict(int)))
        
        for v in ys:
            for u in ys:
                pi[-1][v][u] = max([p * q(v, u, uprime) * e(word, v) for uprime, p in pi[-2][u].items()])
        
        
    pi.append(defaultdict(lambda: defaultdict(int)))
    for y in ys:
        pi[-1]['STOP'][y] = max([p * q('STOP', y, u) for u, p in pi[-2][y].items()])
    
    output = [None] * n
    next_next_y = None
    next_y = None
    
    max_p = 0
    for v in ys:
        for u in ys:
            p = pi[n][v][u] * q('STOP', v, u)
            if p >= max_p:
                next_next_y = v
                next_y = u
                max_p = p
    
    output[n-1] = next_next_y
    output[n-2] = next_y
    
    for k in range(n-1, 1, -1):
        word = sentence[k][0]
        max_p = 0
        best_u = None
        for u in ys:
            p = pi[k][next_y][u] * q(next_next_y, next_y, u) * e(word,next_next_y)
            if p >= max_p:
                max_p = p
                best_u = u
        next_next_y, next_y = next_y, best_u
        output[k-2] = best_u
    return list(zip([word for word, in sentence],output))

def label_with_second_order_hmm(dataset, train_set, k=1, get_second_order_q=get_second_order_q, get_e=get_e):
    ys = set([y for sentence in train_set for (x, y) in sentence])
    q = get_second_order_q(train_set, k=k)
    e = get_e(train_set, k=k)
    return [second_order_hmm(sentence, q, e, ys) for sentence in dataset]

In [ ]:
en_dev_labeled_second_order_hmm = label_with_second_order_hmm(en_dev, en_train)
write_dataset(en_dev_labeled_second_order_hmm, "EN/dev.p4.out")

# EN
## Entity in gold data: 802
## Entity in prediction: 976

## Correct Entity : 562
Entity  precision: 0.5758

Entity  recall: 0.7007

Entity  F: 0.6322

## Correct Entity Type : 411
Entity Type  precision: 0.4211

Entity Type  recall: 0.5125

Entity Type  F: 0.4623

In [ ]:
fr_dev_labeled_second_order_hmm = label_with_second_order_hmm(fr_dev, fr_train)
write_dataset(fr_dev_labeled_second_order_hmm, "FR/dev.p4.out")

# FR
## Entity in gold data: 238
## Entity in prediction: 297

## Correct Entity : 133
Entity  precision: 0.4478

Entity  recall: 0.5588

Entity  F: 0.4972

## Correct Entity Type : 77
Entity Type  precision: 0.2593

Entity Type  recall: 0.3235

Entity Type  F: 0.2879

# Part 5: Design challenge

For the design challenge, we created a total of 3 models, first order HMM with JM smoothing, second order HMM with JM smoothing and third order HMM with JM smoothing. We used JM smoothing as there might be a lot of “holes” in the higher order HMM models as the probabilities get smaller, hence it might be a good idea to interpolate lower order models with higher order ones. It also ensures that unknown words can be handled in a smooth manner. After testing our models on the EN and FR dataset, we chose the first order HMM with JM smoothing as it was having the best results

In [65]:
def get_jm_e(dataset, k=0.99):
    words = set([x for sentence in dataset for (x, y) in sentence])
    count_ys = defaultdict(int)
    count_y_x = defaultdict(lambda  : defaultdict(int))
    count_word = defaultdict(int)
    for sentence in dataset:
        for word, label in sentence:
            count_word[word] += 1
            count_ys[label] += 1
            count_y_x[label][word] += 1
    N = sum([len(sentence) for sentence in dataset])
    def e(x, y):
        return k * count_y_x[y][x]/(count_ys[y]) + (1-k)*k * count_word[x]/N +(1-k)**2 * 1/(len(words) + 1) # + 1 for UNK
    return e

def get_jm_q(dataset, k=0.99):
    qs = defaultdict(lambda  : defaultdict(int))
    count_ys = defaultdict(int)
    count_ys['START'] = len(dataset)
    count_ys['STOP'] = len(dataset)
    for sentence in dataset:
        n = len(sentence)
        for i in range(n):
            current_y = sentence[i][1]
            count_ys[current_y] += 1
            if i == 0:
                qs[current_y]['START'] += 1
            elif i == n-1:
                qs['STOP'][current_y] += 1
                qs[current_y][sentence[i-1][1]] += 1
            else:
                qs[current_y][sentence[i-1][1]] += 1
    N = sum([len(sentence) for sentence in dataset])
    def q(yi, yi_1):
        return k * qs[yi][yi_1]/(count_ys[yi_1]) + (1-k) * count_ys[yi]/N
    return q

def get_second_order_jm_q(dataset, k=0.99):
    qs = defaultdict(lambda  : defaultdict(lambda: defaultdict(int))) # {yi: {yi_1: {yi_2: prob}}}
    count_ys = defaultdict(lambda: defaultdict(int)) # {yi_1: {yi_2: count}}
    count_ys['START']['START'] = len(dataset)
    count_y = defaultdict(int)
    
    for sentence in dataset:
        n = len(sentence)
        for i in range(n):
            current_y = sentence[i][1]
            count_y[current_y] += 1
            if i == 0:
                count_ys[current_y]['START'] += 1
            else:
                prev_y = sentence[i-1][1]
                count_ys[current_y][prev_y] += 1
            
            if i == 0:
                qs[current_y]['START']['START'] += 1
            elif i == 1:
                qs[current_y][sentence[i-1][1]]['START'] += 1
            elif i == n-1:
                qs[current_y][sentence[i-1][1]][sentence[i-2][1]] += 1
                qs['STOP'][current_y][sentence[i-1][1]] += 1
            else:
                qs[current_y][sentence[i-1][1]][sentence[i-2][1]] += 1
    # N = sum([len(sentence) for sentence in dataset])           
    first_order_q = get_jm_q(dataset, k=k)
    def q(yi, yi_1, yi_2):
        # base_rate = count_y[yi]/N
        base_rate = first_order_q(yi, yi_1)
        if count_ys[yi_1][yi_2] == 0:
            return (1-k) * base_rate
        return k * qs[yi][yi_1][yi_2]/(count_ys[yi_1][yi_2]) + (1-k) * base_rate
    return q

# 1st order HMM with JM smoothing

In [ ]:
en_dev_labeled_hmm_jm = label_with_hmm(en_dev, en_train, get_e=get_jm_e, get_q=get_jm_q, k=0.5)
write_dataset(en_dev_labeled_hmm_jm, "EN/dev.p3.jm.0.5.out")

In [ ]:
fr_dev_labeled_hmm_jm = label_with_hmm(fr_dev, fr_train, get_e=get_jm_e, get_q=get_jm_q, k=0.99)
write_dataset(fr_dev_labeled_hmm_jm, "FR/dev.p3.jm.0.99.out")

# 2nd order HMM with JM smoothing

In [ ]:
en_dev_labeled_second_order_hmm_jm = label_with_second_order_hmm(en_dev, en_train, get_e=get_jm_e, get_second_order_q=get_second_order_jm_q, k=0.5)
write_dataset(en_dev_labeled_second_order_hmm_jm, "EN/dev.p4.jm.0.5.out")

In [ ]:
fr_dev_labeled_second_order_hmm_jm = label_with_second_order_hmm(fr_dev, fr_train, get_e=get_jm_e, get_second_order_q=get_second_order_jm_q, k=0.99)
write_dataset(fr_dev_labeled_second_order_hmm_jm, "FR/dev.p4.jm.0.99.out")

# 3rd order HMM with JM smoothing

In [59]:
def get_jm_e(dataset, k=0.99):
    words = set([x for sentence in dataset for (x, y) in sentence])
    count_ys = defaultdict(int)
    count_y_x = defaultdict(lambda  : defaultdict(int))
    count_word = defaultdict(int)
    for sentence in dataset:
        for word, label in sentence:
            count_word[word] += 1
            count_ys[label] += 1
            count_y_x[label][word] += 1
    N = sum([len(sentence) for sentence in dataset])
    def e(x, y):
        return k * count_y_x[y][x]/(count_ys[y]) + (1-k)*k * count_word[x]/N +(1-k)**2 * 1/(len(words) + 1) # + 1 for UNK
    return e

def get_jm_q(dataset, k=0.99):
    qs = defaultdict(lambda  : defaultdict(int))
    count_ys = defaultdict(int)
    count_ys['START'] = len(dataset)
    count_ys['STOP'] = len(dataset)
    for sentence in dataset:
        n = len(sentence)
        for i in range(n):
            current_y = sentence[i][1]
            count_ys[current_y] += 1
            if i == 0:
                qs[current_y]['START'] += 1
            elif i == n-1:
                qs['STOP'][current_y] += 1
                qs[current_y][sentence[i-1][1]] += 1
            else:
                qs[current_y][sentence[i-1][1]] += 1
    N = sum([len(sentence) for sentence in dataset])
    def q(yi, yi_1):
        return k * qs[yi][yi_1]/(count_ys[yi_1]) + (1-k) * count_ys[yi]/N
    return q

def get_second_order_jm_q(dataset, k=0.99):
    qs = defaultdict(lambda  : defaultdict(lambda: defaultdict(int))) # {yi: {yi_1: {yi_2: prob}}}
    count_ys = defaultdict(lambda: defaultdict(int)) # {yi_1: {yi_2: count}}
    count_ys['START']['START'] = len(dataset)
    count_y = defaultdict(int)
    
    for sentence in dataset:
        n = len(sentence)
        for i in range(n):
            current_y = sentence[i][1]
            count_y[current_y] += 1
            if i == 0:
                count_ys[current_y]['START'] += 1
            else:
                prev_y = sentence[i-1][1]
                count_ys[current_y][prev_y] += 1
            
            if i == 0:
                qs[current_y]['START']['START'] += 1
            elif i == 1:
                qs[current_y][sentence[i-1][1]]['START'] += 1
            elif i == n-1:
                qs[current_y][sentence[i-1][1]][sentence[i-2][1]] += 1
                qs['STOP'][current_y][sentence[i-1][1]] += 1
            else:
                qs[current_y][sentence[i-1][1]][sentence[i-2][1]] += 1
    # N = sum([len(sentence) for sentence in dataset])           
    first_order_q = get_jm_q(dataset, k=k)
    def q(yi, yi_1, yi_2):
        # base_rate = count_y[yi]/N
        base_rate = first_order_q(yi, yi_1)
        if count_ys[yi_1][yi_2] == 0:
            return (1-k) * base_rate
        return k * qs[yi][yi_1][yi_2]/(count_ys[yi_1][yi_2]) + (1-k) * base_rate
    return q

def get_third_order_jm_q(dataset, k=0.99):
    qs = defaultdict(lambda  : defaultdict(lambda  : defaultdict(lambda: defaultdict(int)))) # {yi: {yi_1: {yi_2: {yi_3: prob}}}}
    count_ys = defaultdict(lambda :defaultdict(lambda: defaultdict(int))) # {yi_1: {yi_2: {yi_3:count}}}
    count_ys['START']['START']['START'] = len(dataset)
    
    for sentence in dataset:
        n = len(sentence)
        for i in range(n):
            current_y = sentence[i][1]
            if i == 0:
                count_ys[current_y]['START']['START'] += 1
            elif i == 1:
                prev_y = sentence[i-1][1]
                count_ys[current_y][prev_y]['START'] += 1
            else:
                prev_y = sentence[i-1][1]
                prev_2_y = sentence[i-2][1]
                count_ys[current_y][prev_y][prev_2_y] += 1
            
            if i == 0:
                qs[current_y]['START']['START']['START'] += 1
            elif i == 1:
                qs[current_y][sentence[i-1][1]]['START']['START'] += 1
            elif i == 2:
                qs[current_y][sentence[i-1][1]][sentence[i-2][1]]['START'] += 1
            elif i == n-1:
                qs[current_y][sentence[i-1][1]][sentence[i-2][1]][sentence[i-3][1]] += 1
                qs['STOP'][current_y][sentence[i-1][1]][sentence[i-2][1]] += 1
            else:
                qs[current_y][sentence[i-1][1]][sentence[i-2][1]][sentence[i-3][1]] += 1
    second_order_q = get_second_order_jm_q(dataset, k=k)          
    def q(yi, yi_1, yi_2, yi_3):
        base_rate = second_order_q(yi, yi_1, yi_2)
        if qs[yi][yi_1][yi_2][yi_3] == 0:
            return (1-k) * base_rate
        return k * qs[yi][yi_1][yi_2][yi_3]/(count_ys[yi_1][yi_2][yi_3]) + (1-k) * base_rate
    return q


def label_with_third_order_hmm(dataset, train_set, k=1, get_third_order_q=get_third_order_jm_q, get_e=get_e):
    ys = set([y for sentence in train_set for (x, y) in sentence])
    q = get_third_order_jm_q(train_set, k=k)
    e = get_e(train_set, k=k)
    return [third_order_hmm(sentence, q, e, ys) for sentence in dataset]

In [60]:
def third_order_hmm(sentence, q, e, ys):
    pi = [defaultdict(lambda: defaultdict(lambda: defaultdict(int))), defaultdict(lambda: defaultdict(lambda: defaultdict(int))), defaultdict(lambda: defaultdict(lambda: defaultdict(int)))] # array of {w: {v: {u: prob}}}, w being most recent, yi
    pi[0]['START']['START']['START'] = 1
    
    # 1st layer
    for y in ys:
        pi[1][y]['START']['START'] = q(y, 'START', 'START', 'START') * e(sentence[0][0], y)
    
    # 2nd layer
    for w in ys:
        for v in ys:
            pi[2][w][v]['START'] = max([p * q(w, v, 'START', vprime) * e(sentence[1][0], w) for vprime, p in pi[1][v]['START'].items()])
            
    n = len(sentence)
    
    for i in range(1, n):
        word = sentence[i][0]
        
        pi.append(defaultdict(lambda: defaultdict(lambda: defaultdict(int))))
        
        for w in ys:
            for v in ys:
                for u in ys:
                    pi[-1][w][v][u] = max([p * q(w, v, u, uprime) * e(word, w) for uprime, p in pi[-2][v][u].items()])
        
        
    pi.append(defaultdict(lambda: defaultdict(lambda: defaultdict(int))))
    for w in ys:
        for v in ys:
            pi[-1]['STOP'][w][v] = max([p * q('STOP', w, v, u) for u, p in pi[-2][w][v].items()])
    
    output = [None] * n
    next_next_next_y = None
    next_next_y = None
    next_y = None
    
    max_p = 0
    for w in ys:
        for v in ys:
            for u in ys:
                p = pi[n][w][v][u] * q('STOP', w, v, u)
                if p >= max_p:
                    next_next_next_y = w
                    next_next_y = v
                    next_y = u
                    max_p = p
    
    output[n-1] = next_next_next_y
    output[n-2] = next_next_y
    output[n-3] = next_y
    
    for k in range(n-1, 1, -1):
        word = sentence[k][0]
        max_p = 0
        best_u = None
        for u in ys:
            p = pi[k][next_next_y][next_y][u] * q(next_next_next_y, next_next_y, next_y, u) * e(word,next_next_next_y)
            if p >= max_p:
                max_p = p
                best_u = u
        next_next_next_y, next_next_y, next_y = next_next_y, next_y, best_u
        output[k-3] = best_u
    return list(zip([word for word, in sentence],output))

In [61]:
en_dev_labeled_3rd_order_hmm_jm = label_with_third_order_hmm(en_dev, en_train, get_e=get_jm_e, get_third_order_q=get_third_order_jm_q, k=0.99)
write_dataset(en_dev_labeled_3rd_order_hmm_jm, "EN/dev.p5.jm.0.99.out")

In [63]:
fr_dev_labeled_3rd_order_hmm_jm = label_with_third_order_hmm(fr_dev, fr_train, get_e=get_jm_e, get_third_order_q=get_third_order_jm_q, k=0.99)
write_dataset(fr_dev_labeled_3rd_order_hmm_jm, "FR/dev.p5.jm.0.99.out")